# Генерация данных для volume и percent

In [1]:
import random, re
from typing import List, Tuple, Optional
import pandas as pd

# ===== утилиты =====
_WS_RE = re.compile(r"\S+")
def ws_offsets(text: str):
    return [(m.group(0), m.start(), m.end()) for m in _WS_RE.finditer(text or "")]

def bio_init(n: int) -> List[str]:
    return ["O"] * n

def paint(y: List[str], s: int, e: int, tag: str):
    y[s] = f"B-{tag}"
    for i in range(s+1, e+1):
        y[i] = f"I-{tag}"

def to_text_and_ann(tokens: List[str], labels: List[str]) -> Tuple[str, str]:
    text = " ".join(tokens)
    triples = []
    for (tok, s, e), lab in zip(ws_offsets(text), labels):
        triples.append(f"({s}, {e}, '{lab}')")
    return text, "[" + ", ".join(triples) + "]"

# ===== словари =====
# бренды: короткие, реалистичные; оставляю латиницу — это полезно для нуджа BRAND
BRANDS = [
    "Danone", "Prostokvashino", "Oatly", "Heinz", "Nestle",
    "Milka", "Mirel", "Activia", "Rama", "Chudo", "Vkusnoteevo"
]
# типы (категории/наименования товара) — русские, иногда многословные
TYPES = [
    "йогурт", "кефир", "молоко", "сыр", "творог", "сливочное масло",
    "растительное масло", "печенье", "батончик", "шоколад",
    "лимонад", "сок", "соус", "кетчуп", "майонез", "творожный сыр"
]
# вкусы/описания для лёгкого контекста
FLAVORS = ["клубника", "ваниль", "шоколад", "персик", "без сахара", "классический", "лайт"]

# единицы — только русские (включая склонения и частые опечатки)
UNITS_SHORT = ["мл", "л", "г", "гр", "кг", "шт"]
UNITS_FULL  = [
    "миллилитр","миллилитра","миллилитров",
    "литр","литра","литров",
    "грамм","грамма","граммов",
    "килограмм","килограмма","килограммов",
    "штука","штуки","штук"
]
UNITS_TYPO  = ["милилитр","милилитра","милилитров","киллограмм","киллограмма","киллограммов","грам"]
UNITS = UNITS_SHORT + UNITS_FULL + UNITS_TYPO
COMPACT_UNITS = {"мл","л","г","гр","кг","шт"}  # допустимо слитно: 500г, 1,5л

PCT_WORDS = ["%", "процент", "процента", "процентов", "проценты", "проц", "проц."]
ATTR_PCT  = ["жирность","содержание сахара","сахар","какао","белок"]

# ===== генерация блоков =====
def _rand_number():
    if random.random() < 0.55:
        return str(random.choice([100,150,200,250,300,330,400,450,500,700,750,900,1000,1500,2000]))
    val = random.choice([0.2,0.25,0.3,0.33,0.5,0.75,1.0,1.5,2.0,2.5,3.2,5.0,7.5,10.0,12.0,15.0,20.0,25.0])
    s = f"{val}"
    return s if random.random() < 0.5 else s.replace(".", ",")

def block_volume():
    num, unit = _rand_number(), random.choice(UNITS)
    if unit in COMPACT_UNITS and random.random() < 0.30:
        tok = num + unit
        return [tok], [("VOLUME", 0, 0)]
    return [num, unit], [("VOLUME", 0, 1)]

def block_percent():
    num, pct = _rand_number(), random.choice(PCT_WORDS)
    toks, spans = [], []
    if random.random() < 0.55:
        toks.extend(random.choice(ATTR_PCT).split())
    if random.random() < 0.45:
        tok = num + ("" if pct == "%" else "") + pct   # 3,2%
        toks.append(tok); spans.append(("PERCENT", len(toks)-1, len(toks)-1))
    else:
        toks += [num, pct]; spans.append(("PERCENT", len(toks)-2, len(toks)-1))
    return toks, spans

def make_sample_row(
    brand: Optional[str],
    typ: Optional[str],
    need_volume: bool,
    need_percent: bool,
    add_flavor_prob: float = 0.4,
):
    tokens: List[str] = []
    spans: List[Tuple[str,int,int]] = []

    # TYPE (максимум один)
    if typ:
        tks = typ.split()
        s = len(tokens); tokens += tks; e = len(tokens)-1
        spans.append(("TYPE", s, e))

    # BRAND (максимум один)
    if brand:
        bks = brand.split()
        s = len(tokens); tokens += bks; e = len(tokens)-1
        spans.append(("BRAND", s, e))

    # немного контекста
    if random.random() < add_flavor_prob:
        tokens += random.choice(FLAVORS).split()

    # измерения
    blocks = []
    if need_volume:  blocks.append(block_volume())
    if need_percent: blocks.append(block_percent())
    random.shuffle(blocks)
    for btoks, bspans in blocks:
        shift = len(tokens); tokens += btoks
        for tag, s_rel, e_rel in bspans:
            spans.append((tag, shift + s_rel, shift + e_rel))

    # BIO
    y = bio_init(len(tokens))
    for tag, s, e in spans:
        paint(y, s, e, tag)

    return to_text_and_ann(tokens, y)

def generate_fixed_brand_type(
    n_rows: int = 2000,
    seed: int = 123,
    ensure_both_frac: float = 0.6,  # доля строк с ОБОИМИ: VOLUME и PERCENT
    p_brand: float = 1.0,           # вероятность вставить один BRAND
    p_type: float  = 1.0,           # вероятность вставить один TYPE
):
    random.seed(seed)
    rows = []
    need_both = int(n_rows * ensure_both_frac)

    while len(rows) < n_rows:
        brand = random.choice(BRANDS) if random.random() < p_brand else None
        typ   = random.choice(TYPES)  if random.random() < p_type  else None

        # минимум одна измерительная сущность
        if need_both > 0:
            need_volume = True; need_percent = True
        else:
            need_volume  = random.random() < 0.9
            need_percent = random.random() < 0.8
            if not (need_volume or need_percent):
                need_volume = True

        text, ann = make_sample_row(brand, typ, need_volume, need_percent)
        rows.append((text, ann))
        if need_volume and need_percent and need_both > 0:
            need_both -= 1

    return pd.DataFrame(rows, columns=["sample","annotation"]).drop_duplicates().reset_index(drop=True)

In [2]:
df_sync = generate_fixed_brand_type(
    n_rows=1400,
    seed=123,
    ensure_both_frac=0.6,
    p_brand=1.0,
    p_type=1.0
)

In [3]:
df_sync.to_csv(r"./.data/sync_data.csv", sep=";", index=False)

In [4]:
df_train = pd.read_csv(r"./.data/input/train.csv", sep = ";", encoding="utf-8")

In [5]:
df_mix = pd.concat([df_train, df_sync], ignore_index=True)

In [6]:
df_mix.shape

(28849, 2)

In [7]:
df_mix.to_csv(r"./.data/train.csv", sep=";", index=False)

# Генерация негативных данных 

In [8]:
import random
import re
from typing import List, Tuple
import pandas as pd

# Параметры генерации
SEED = 123
N_SAMPLES = 3000          # сколько строк синтетики сделать
DEDUP = True              # убирать дубликаты
P_MULTI_TYPE = 0.25       # доля случаев с B-TYPE → I-TYPE
P_BRAND_CASE = 0.10       # доля случаев с B-BRAND + O
P_ALL_O = 0.20            # доля полностью O-строк (бессмысленные слова)
MAX_TOKENS = 5            # верхняя длина запроса (в токенах)

random.seed(SEED)


In [9]:
# Базовые "типовые" слова (TYPE). Коротко и по-русски.
TYPES_BASE = [
    "гречка","макароны","булочки","жидкость","кабачки","курица","перловка","помидор",
    "пюре","салаты","соль","творожки","уголь","хлопья","шаурма","вода","йогурт",
    "печенье","рис","кофе","чай","сыр","колбаса","кетчуп","сосиски","лапша","мука",
]

# Возможные вторые слова TYPE (I-TYPE), когда тип многословный
TYPE_MODS = [
    "бедро","филе","грудка","нарезка","батон","рамен","спагетти","детский","фермерский","столовый"
]

# Предлоги/служебные (часто O)
O_PREPS = ["с","в","для","без","на","из","по"]

# Объекты/прилагательные под O-хвост (реалистичные для продуктовых запросов)
O_TAIL = [
    "вкусом","солью","глютена","аджике","пакетах","банке","изюмом","кошек","завтрак",
    "шашлыка","отрубями","чесноком","перцем","детей","тебе","всей","семье","собаки","собак",
]

# Слоговая база для псевдо-брендов и "абракадабры"
SYL = ["ра","мо","ко","на","ли","ма","бе","ри","лу","ша","ви","та","зо","не","фа","ду","хе","ки","то","са"]

RUS_LETTERS = list("абвгдеёжзийклмнопрстуфхцчшщьыъэюя")

def add_noise(word: str, p: float = 0.25) -> str:
    """Лёгкие опечатки: drop / swap / substitute (по одной-двум операциям)."""
    if random.random() > p or len(word) < 3:
        return word
    s = list(word)
    op = random.choice(["drop", "swap", "sub"])
    if op == "drop" and len(s) > 3:
        i = random.randrange(len(s))
        del s[i]
    elif op == "swap" and len(s) > 3:
        i = random.randrange(len(s)-1)
        s[i], s[i+1] = s[i+1], s[i]
    else:  # substitute
        i = random.randrange(len(s))
        s[i] = random.choice(RUS_LETTERS)
    out = "".join(s)
    # иногда вторая маленькая опечатка
    if random.random() < 0.15 and len(out) > 3:
        return add_noise(out, p=1.0)
    return out

def synth_brand() -> str:
    """Псевдо-бренд (без референсов к существующим): 2–3 слога."""
    k = random.choice([2,2,3])
    return "".join(random.choice(SYL).capitalize() if i == 0 else random.choice(SYL) for i in range(k))

def gibberish() -> str:
    """Бессмысленное слово (для полностью O-строк)."""
    k = random.choice([2,3])
    w = "".join(random.choice(SYL) for _ in range(k))
    # иногда добавим опечатку
    return add_noise(w, p=0.4)


In [10]:
Tag = str

def make_type_tokens() -> Tuple[List[str], List[Tag]]:
    """B-TYPE (+ опц. I-TYPE) + O-хвост (предлоги/слова)."""
    head = add_noise(random.choice(TYPES_BASE), p=0.35)
    tokens = [head]
    tags   = ["B-TYPE"]
    if random.random() < P_MULTI_TYPE:
        mod = add_noise(random.choice(TYPE_MODS), p=0.35)
        tokens.append(mod)
        tags.append("I-TYPE")
    # хвост из O: предлог + слово (иногда два слова)
    if random.random() < 0.85:
        prep = random.choice(O_PREPS)
        obj  = add_noise(random.choice(O_TAIL), p=0.35)
        tokens.extend([prep, obj])
        tags.extend(["O", "O"])
        if random.random() < 0.25:
            obj2 = add_noise(random.choice(O_TAIL), p=0.35)
            tokens.append(obj2)
            tags.append("O")
    # ограничим длину
    tokens = tokens[:MAX_TOKENS]
    tags   = tags[:len(tokens)]
    return tokens, tags

def make_brand_tokens() -> Tuple[List[str], List[Tag]]:
    """B-BRAND + 1–2 O-слова (в духе «сейчас», «новый», etc.)."""
    brand = add_noise(synth_brand(), p=0.15)
    tail_opts = ["сейчас", "новый", "акция", "тебе", "рядом"]
    toks = [brand]
    tg   = ["B-BRAND"]
    toks.append(add_noise(random.choice(tail_opts), p=0.35)); tg.append("O")
    if random.random() < 0.35:
        toks.append(add_noise(random.choice(O_TAIL), p=0.35)); tg.append("O")
    return toks[:MAX_TOKENS], tg[:MAX_TOKENS]

def make_all_o_tokens() -> Tuple[List[str], List[Tag]]:
    """Полностью O — 1–3 бессмысленных слова/ошибок."""
    n = random.choice([1,2,3])
    toks = [gibberish() for _ in range(n)]
    return toks, ["O"]*n

def tokens_to_spans(tokens: List[str], tags: List[Tag]) -> Tuple[str, List[Tuple[int,int,str]]]:
    """
    Из токенов и BIO-меток делаем строку и список спанов (start,end,label),
    где label — 'O' или 'B-XXX'/'I-XXX' как в твоём train.
    """
    text = " ".join(tokens)
    spans = []
    cur = 0
    for i, (tok, tag) in enumerate(zip(tokens, tags)):
        start = cur
        end = start + len(tok)
        spans.append((start, end, tag))
        cur = end + 1  # пробел
    return text, spans


In [11]:
def sample_case() -> Tuple[str, List[Tuple[int,int,str]]]:
    r = random.random()
    if r < P_ALL_O:
        toks, tags = make_all_o_tokens()
    elif r < P_ALL_O + P_BRAND_CASE:
        toks, tags = make_brand_tokens()
    else:
        toks, tags = make_type_tokens()
    return tokens_to_spans(toks, tags)

def build_negative_df(n: int, dedup: bool = True) -> pd.DataFrame:
    seen = set()
    data = []
    attempts = 0
    while len(data) < n and attempts < n * 20:
        attempts += 1
        s, spans = sample_case()
        if not s.strip():
            continue
        if dedup and s in seen:
            continue
        seen.add(s)
        data.append((s, str(spans)))  # строковое представление списка спанов, как в твоём CSV
    if len(data) < n:
        print(f"[warn] набрано {len(data)} из {n} (остальное отфильтровано дублями/пустыми).")
    df = pd.DataFrame(data, columns=["sample", "annotation"])
    return df

df_synth_neg = build_negative_df(N_SAMPLES, dedup=DEDUP)
df_synth_neg.head(12)


,sample,annotation
0,луфамо,"[(0, 6, 'O')]"
1,римара саоз,"[(0, 6, 'O'), (7, 11, 'O')]"
2,булочки бедор по шшалыка,"[(0, 7, 'B-TYPE'), (8, 13, 'I-TYPE'), (14, 16,..."
3,солю по банке всей,"[(0, 4, 'B-TYPE'), (5, 7, 'O'), (8, 13, 'O'), ..."
4,кофазо рааето,"[(0, 6, 'O'), (7, 13, 'O')]"
5,уголь,"[(0, 5, 'B-TYPE')]"
6,вода без банче пакетах,"[(0, 4, 'B-TYPE'), (5, 8, 'O'), (9, 14, 'O'), ..."
7,ненеа коне,"[(0, 5, 'O'), (6, 10, 'O')]"
8,угобь по осбаки,"[(0, 5, 'B-TYPE'), (6, 8, 'O'), (9, 15, 'O')]"
9,хцопья из солью,"[(0, 6, 'B-TYPE'), (7, 9, 'O'), (10, 15, 'O')]"


In [12]:
from collections import Counter
import ast

def infer_pattern(row) -> str:
    try:
        spans = ast.literal_eval(row["annotation"])
    except Exception:
        return "?"
    tags = [t for _,_,t in spans]
    if all(t == "O" for t in tags):
        return "ALL_O"
    if tags and tags[0].startswith("B-BRAND"):
        return "BRAND+O"
    if len(tags) > 1 and tags[0] == "B-TYPE" and tags[1] == "I-TYPE":
        return "TYPE+I+O"
    return "TYPE+O"

pat = df_synth_neg.head(50).copy()
pat["pattern"] = pat.apply(infer_pattern, axis=1)
display(pat[["sample","annotation","pattern"]].head(20))

cnt = Counter(df_synth_neg.sample(min(1000, len(df_synth_neg)), random_state=SEED).apply(infer_pattern, axis=1))
print(cnt)


,sample,annotation,pattern
0,луфамо,"[(0, 6, 'O')]",ALL_O
1,римара саоз,"[(0, 6, 'O'), (7, 11, 'O')]",ALL_O
2,булочки бедор по шшалыка,"[(0, 7, 'B-TYPE'), (8, 13, 'I-TYPE'), (14, 16,...",TYPE+I+O
3,солю по банке всей,"[(0, 4, 'B-TYPE'), (5, 7, 'O'), (8, 13, 'O'), ...",TYPE+O
4,кофазо рааето,"[(0, 6, 'O'), (7, 13, 'O')]",ALL_O
5,уголь,"[(0, 5, 'B-TYPE')]",TYPE+O
6,вода без банче пакетах,"[(0, 4, 'B-TYPE'), (5, 8, 'O'), (9, 14, 'O'), ...",TYPE+O
7,ненеа коне,"[(0, 5, 'O'), (6, 10, 'O')]",ALL_O
8,угобь по осбаки,"[(0, 5, 'B-TYPE'), (6, 8, 'O'), (9, 15, 'O')]",TYPE+O
9,хцопья из солью,"[(0, 6, 'B-TYPE'), (7, 9, 'O'), (10, 15, 'O')]",TYPE+O


Counter({'TYPE+O': 487, 'ALL_O': 218, 'TYPE+I+O': 175, 'BRAND+O': 120})


In [13]:
OUT_PATH = ".data/synth_negative_products.csv"
df_synth_neg.to_csv(OUT_PATH, index=False, sep=";")
print(f"[info] saved -> {OUT_PATH}  rows={len(df_synth_neg)}")


[info] saved -> .data/synth_negative_products.csv  rows=3000
